### Copyright
**These notebooks, authored by Chun Fu and Hussain Kazmi, are accompanying material to the first Buildsys tutorial on load forecasting at building and urban scale. They are licensed under the AGPL License; you may not use this file except in compliance with the License. Notebooks are distributed on an “AS IS” BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.**

# Notebook 0: Data preparation

This notebook will help you prepare a dataset for modeling.

The original dataset consists of 112 separate CSV files (one file for one block), which will be combined into a single file and stored in a pivot format to conserve storage space, and reduce loading times. The 

To get different aggregation levels of energy demand, we will export three files at the following aggregation levels:

- City level
- Block level
- House level

We begin with importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import tqdm
import os

Next, let us define the path to the dataset from where we will load it.

In [ ]:
path_dataset=r'C:\Users\patri\Buildsys_Tutorial\dataset'
path_dataset_meters=r'C:\Users\patri\Buildsys_Tutorial\dataset\halfhourly_dataset\halfhourly_dataset'

os.listdir(path_dataset)

['acorn_details.csv',
 'daily_dataset.csv',
 'darksky_parameters_documentation.html',
 'elec_block.csv',
 'elec_house.csv',
 'elec_nation.csv',
 'halfhourly_dataset',
 'informations_households.csv',
 'uk_bank_holidays.csv',
 'weather_daily_darksky.csv',
 'weather_hourly_darksky.csv']

At this point, we load the dataset from the defined path, and print a snapshot.

In [ ]:
halfhourly_dataset = []

for file_name in tqdm.tqdm(os.listdir(path_dataset_meters)):
    df_temp = pd.read_csv(os.path.join(path_dataset_meters, file_name),index_col='tstp',parse_dates=True, low_memory=False)
    df_temp['file_name'] = file_name.split('.')[0]
    df_temp = df_temp.replace('Null', np.nan).dropna()
    df_temp['energy(kWh/hh)'] = df_temp['energy(kWh/hh)'].astype('float')
    halfhourly_dataset.append(df_temp)
    
halfhourly_dataset = pd.concat(halfhourly_dataset, axis=0)
halfhourly_dataset

100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [04:57<00:00,  2.66s/it]


,LCLid,energy(kWh/hh),file_name
tstp,,,
2012-10-12 00:30:00,MAC000002,0.000,block_0
2012-10-12 01:00:00,MAC000002,0.000,block_0
2012-10-12 01:30:00,MAC000002,0.000,block_0
2012-10-12 02:00:00,MAC000002,0.000,block_0
2012-10-12 02:30:00,MAC000002,0.000,block_0
...,...,...,...
2014-02-27 22:00:00,MAC005019,0.129,block_99
2014-02-27 22:30:00,MAC005019,0.095,block_99
2014-02-27 23:00:00,MAC005019,0.061,block_99


The datasets contain timestamps which show that data is sampled at a half hourly interval. The data also contains the building ID as well as the energy demand, which we are interested in modelling. 

Let's take a closer look at the dataframe next.

In [ ]:
halfhourly_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 167811461 entries, 2012-10-12 00:30:00 to 2014-02-28 00:00:00
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   LCLid           object 
 1   energy(kWh/hh)  float64
 2   file_name       object 
dtypes: float64(1), object(2)
memory usage: 5.0+ GB


We can now use the pivot_table command to create three separate dataframes (i.e. at different aggregation levels), and write them to csv files on disk.

In [ ]:
df_house = halfhourly_dataset.pivot_table(index='tstp',values='energy(kWh/hh)',columns='LCLid')
df_block = halfhourly_dataset.pivot_table(index='tstp',values='energy(kWh/hh)',columns='file_name')
df_city = halfhourly_dataset.pivot_table(index='tstp',values='energy(kWh/hh)')

In [ ]:
df_city.to_csv(os.path.join(path_dataset,'elec_city.csv'))
df_block.to_csv(os.path.join(path_dataset,'elec_block.csv'))
df_house.to_csv(os.path.join(path_dataset,'elec_house.csv'))

In [ ]:
#Ram-saving version

#df_output = halfhourly_dataset.pivot_table(index='tstp',values='energy(kWh/hh)',columns='LCLid')
#df_output.to_csv('elec_house.csv')

#df_output = halfhourly_dataset.groupby(['tstp', 'file_name'])['energy(kWh/hh)'].mean().unstack()
#df_output.to_csv('elec_block.csv')

#df_output = halfhourly_dataset.pivot_table(index='tstp',values='energy(kWh/hh)')
#df_output.to_csv('elec_city.csv')